In [ ]:
import altair as alt
import polars as pl

from alltime_athletics_viz.io import import_running_only_events

%load_ext autoreload
%autoreload 2
alt.data_transformers.disable_max_rows()

In [ ]:
df = import_running_only_events()

In [ ]:
world_records = (
    df.groupby("sex", "event", maintain_order=True).first().sort("sex", "distance")
)
world_records

In [ ]:
comparison_women_men = (
    world_records.filter(
        pl.col("event").is_in(["100m hurdles", "110m hurdles"]).is_not()
    )
    .with_columns(
        (
            100
            * pl.col("result seconds").head(2).last().over("event")
            / pl.col("result seconds").head(2).first().over("event")
            - 100
        ).alias("Delta WR time Women vs Men in Percent")
    )
    .filter(pl.col("sex") == "women")
    .select(
        "event",
        "name",
        "result",
        "result seconds",
        "Delta WR time Women vs Men in Percent",
        "distance type",
    )
    .sort("result seconds")
)
comparison_women_men

In [ ]:
# legend_selection = alt.selection_point(fields=["unit"], bind="legend")
chart = (
    alt.Chart(comparison_women_men)
    # .mark_line(clip=True)
    .mark_bar()
    # .mark_point(filled=True, size=10, clip=True)
    # .mark_line(clip=True, interpolate="step-after",point=True)
    .encode(
        x=alt.X("event:N")
        .scale(zero=False)
        .sort(comparison_women_men["event"].to_list()),
        y=alt.Y("Delta WR time Women vs Men in Percent:Q").scale(zero=False),
        color=(
            alt.Color("distance type:N")
            .scale(zero=False, scheme="dark2")
            .legend(columns=1, symbolLimit=0, labelLimit=0)
        ),
        # detail=alt.Detail(":N"),
        # row=alt.Row(":N",spacing=50),
        # shape=alt.Shape(":N"),
        # opacity=alt.condition(legend_selection, alt.value(1), alt.value(0.1)),
    ).properties(width=1400, height=700)
    # .resolve_scale(x="independent")
    # .resolve_scale(y="independent")
    # .add_params(legend_selection)
    # .properties(title={"text": "TITLE GOES HERE", **Footer().subtitle()})
    # .configure_title(anchor="start")
    # .configure_axisX(titleAlign="right")
    # .configure_axisY(titleY=100, titleAlign="center", titleAngle=-90, titleX=-50)
    # .interactive()
)
chart